<a href="https://colab.research.google.com/github/mdcdxcvi/imersao-dados-desafio-final/blob/main/Notebooks/Vin%C3%ADcius_O_Romano_Silva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conjunto de dados 

In [177]:
import pandas as pd


# Carrega as duas bases de dados
dados_experimento = pd.read_csv('https://github.com/mdcdxcvi/imersao-dados-desafio-final/blob/main/Dados/dados_experimentos.zip?raw=true', compression='zip')
dados_resultados = pd.read_csv('https://github.com/mdcdxcvi/imersao-dados-desafio-final/blob/main/Dados/dados_resultados.csv?raw=true')

In [178]:
# Tratamento das colunas do DataFrame dados_experimento
dados_experimento.rename(columns={'droga':'composto'}, inplace=True)


# Tratamento do DataFrame dados_resultados
def seleciona_reacoes(linha_dados: pd.core.series.Series) -> list:
    """Limpa as reações não registradas em cada coluna.
    
    argumentos:
    linha_dados -- linha do df em formato série
    """
    return linha_dados.to_frame().dropna(axis=0).index.values.tolist()

reacoes = pd.DataFrame(
    dados_resultados.iloc[:,1:][dados_resultados.iloc[:,1:]==1].apply(
        lambda x: seleciona_reacoes(x),
        axis='columns',
    ),
    columns=['reacao_observada'],
)
reacoes['id'] = dados_resultados.iloc[:,0]


# Une as duas bases de dados
dados = pd.merge(
    dados_experimento,
    reacoes,
    on='id'
)

In [179]:
import plotly.express as px
from ipywidgets import interactive


# Gráfico interativo para visualização das alterações nos indicadores dado o index da base de dados.
def plot_vizualizacao(indice: int = 256) -> None:
    """Gráfico das alterações em relação ao index da entrada nos dados.
    
    argumentos:
    indice -- index da entrada nos dados (padrão: 256)
    """
    exp_genicas_e_alteracoes = dados.iloc[indice].loc['g-0':'c-99'].to_frame().T
    fig = px.bar(
        dados.iloc[indice].loc['g-0':'c-99'],
        x=exp_genicas_e_alteracoes.columns,
        y=exp_genicas_e_alteracoes.values.tolist()[0],
        title=f'{dados.id[indice]}; tratamento: {dados["tratamento"][indice]}; tempo: {dados.tempo[indice]}; dose: {dados.dose[indice]}; composto: {dados.composto[indice]}.',
        color=exp_genicas_e_alteracoes.values.tolist()[0],
        color_continuous_scale=px.colors.sequential.Bluered_r,
        labels={
            'x': 'Indicador',
            'y': 'Alteração',
            'color': 'cor',
        },
        width=1200,
        range_y=[-10,10],
    )

    reacao = dados.reacao_observada[indice]
    if reacao == []:
        display(pd.DataFrame(data=['Nenhuma'],columns=['Reação']).T)
    elif len(reacao) == 1:
        display(pd.DataFrame(data=reacao,columns=['Reação']).T)
    else:
        display(pd.DataFrame(data=reacao,columns=['Reações']).T)

    fig.show()
 

interactive_plot = interactive(plot_vizualizacao, indice=(0, dados.shape[0] - 1))
output = interactive_plot.children[-1]
interactive_plot

interactive(children=(IntSlider(value=256, description='indice', max=23813), Output()), _dom_classes=('widget-…

# Machine Learning

In [180]:
import copy


# Tratando o DataFrame dados_resultados
dados_resultados_tratados_completos = pd.DataFrame(data=dados_resultados['id'])

for alvo_mec_acao in dados_resultados.iloc[:,1:].columns:
    if alvo_mec_acao.split('_')[-1] in ['inhibitor', 'agonist', 'antagonist', 'activator']:
        coluna_alvo = '_'.join(alvo_mec_acao.split('_')[:-1])
        alvo_mec_acao_alvo = alvo_mec_acao.split('_')[-1]
        if coluna_alvo not in dados_resultados_tratados_completos.columns:
            dados_resultados_tratados_completos[coluna_alvo] = 0

        dados_resultados_tratados_completos.loc[dados_resultados[alvo_mec_acao] == 1, coluna_alvo] = alvo_mec_acao_alvo

    elif alvo_mec_acao not in dados_resultados_tratados_completos.columns:
        dados_resultados_tratados_completos[alvo_mec_acao] = 0
        dados_resultados_tratados_completos.loc[dados_resultados[alvo_mec_acao] == 1, alvo_mec_acao] = 'apresenta'

dados_resultados_tratados_completos.replace(to_replace={0:'NA'}, inplace=True)
dados_resultados_tratados = copy.copy(dados_resultados_tratados_completos)

In [181]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split


def criar_listas_vazias(quantas: int) -> list:
    """Cria uma quantidade de lista de listas vazias.
      
    argumentos:
    quantas -- quantas listas vazias devem ser criadas
    """
    return [ [] for _ in range(len(quantas)) ]


profund = [2, 5]
alvos_mec_acao_observados = dados_resultados_tratados.loc[:,'5-alpha_reductase':'adenylyl_cyclase'].columns
teste_g = pd.DataFrame(index=range(profund[0],profund[1]))
teste_c = pd.DataFrame(index=range(profund[0],profund[1]))
teste_gc = pd.DataFrame(index=range(profund[0],profund[1]))
treino_g = pd.DataFrame(index=range(profund[0],profund[1]))
treino_c = pd.DataFrame(index=range(profund[0],profund[1]))
treino_gc = pd.DataFrame(index=range(profund[0],profund[1]))

to_drop = []
for alvo_mec_acao_obs in alvos_mec_acao_observados:
    if min(dados_resultados_tratados[alvo_mec_acao_obs].value_counts()) < 2:
        to_drop.append(alvo_mec_acao_obs)

alvos_mec_acao_observados = alvos_mec_acao_observados.drop(to_drop)
dados_resultados_tratados.drop(to_drop, inplace=True)


# Desenvolvimento e treino do modelo
modelos_g = criar_listas_vazias(alvos_mec_acao_observados)
modelos_c = criar_listas_vazias(alvos_mec_acao_observados)
modelos_gc = criar_listas_vazias(alvos_mec_acao_observados)
X_treinos_g = criar_listas_vazias(alvos_mec_acao_observados)
X_treinos_c = criar_listas_vazias(alvos_mec_acao_observados)
X_treinos_gc = criar_listas_vazias(alvos_mec_acao_observados)
X_testes_g = criar_listas_vazias(alvos_mec_acao_observados)
X_testes_c = criar_listas_vazias(alvos_mec_acao_observados)
X_testes_gc = criar_listas_vazias(alvos_mec_acao_observados)
y_treinos_g = criar_listas_vazias(alvos_mec_acao_observados)
y_treinos_c = criar_listas_vazias(alvos_mec_acao_observados)
y_treinos_gc = criar_listas_vazias(alvos_mec_acao_observados)
y_testes_g = criar_listas_vazias(alvos_mec_acao_observados)
y_testes_c = criar_listas_vazias(alvos_mec_acao_observados)
y_testes_gc = criar_listas_vazias(alvos_mec_acao_observados)

for alvo_mec_acao_obs in alvos_mec_acao_observados:
    X_g = dados_experimento.loc[:,'g-0':'g-771'].select_dtypes('float64')
    X_c = dados_experimento.loc[:,'c-0':'c-99'].select_dtypes('float64')
    X_gc = dados_experimento.loc[:,'g-0':'c-99'].select_dtypes('float64')
    y = dados_resultados_tratados[alvo_mec_acao_obs].to_frame()
    X_treino_g, X_teste_g, y_treino_g, y_teste_g = train_test_split(
        X_g,
        y,
        test_size=0.2,
        stratify=y,
        random_state=0,
    )
    X_treino_c, X_teste_c, y_treino_c, y_teste_c = train_test_split(
        X_c,
        y,
        test_size=0.2,
        stratify=y,
        random_state=0,
    )
    X_treino_gc, X_teste_gc, y_treino_gc, y_teste_gc = train_test_split(
        X_gc,
        y,
        test_size=0.2,
        stratify=y,
        random_state=0,
    )
    X_treinos_g[dados_resultados_tratados.columns.get_loc(alvo_mec_acao_obs)-1].append(X_treino_g)
    X_treinos_c[dados_resultados_tratados.columns.get_loc(alvo_mec_acao_obs)-1].append(X_treino_c)
    X_treinos_gc[dados_resultados_tratados.columns.get_loc(alvo_mec_acao_obs)-1].append(X_treino_gc)
    X_testes_g[dados_resultados_tratados.columns.get_loc(alvo_mec_acao_obs)-1].append(X_teste_g)
    X_testes_c[dados_resultados_tratados.columns.get_loc(alvo_mec_acao_obs)-1].append(X_teste_c)
    X_testes_gc[dados_resultados_tratados.columns.get_loc(alvo_mec_acao_obs)-1].append(X_teste_gc)
    y_treinos_g[dados_resultados_tratados.columns.get_loc(alvo_mec_acao_obs)-1].append(y_treino_g)
    y_treinos_c[dados_resultados_tratados.columns.get_loc(alvo_mec_acao_obs)-1].append(y_treino_c)
    y_treinos_gc[dados_resultados_tratados.columns.get_loc(alvo_mec_acao_obs)-1].append(y_treino_gc)
    y_testes_g[dados_resultados_tratados.columns.get_loc(alvo_mec_acao_obs)-1].append(y_teste_g)
    y_testes_c[dados_resultados_tratados.columns.get_loc(alvo_mec_acao_obs)-1].append(y_teste_c)
    y_testes_gc[dados_resultados_tratados.columns.get_loc(alvo_mec_acao_obs)-1].append(y_teste_gc)
    teste_g[alvo_mec_acao_obs] = 0
    teste_c[alvo_mec_acao_obs] = 0
    teste_gc[alvo_mec_acao_obs] = 0
    treino_g[alvo_mec_acao_obs] = 0
    treino_c[alvo_mec_acao_obs] = 0
    treino_gc[alvo_mec_acao_obs] = 0

    for i in range(profund[0],profund[1]):
        modelo_arvore = DecisionTreeClassifier(max_depth=i)
        modelo_arvore.fit(X_treino_g, y_treino_g)
        modelos_g[i-profund[0]].append(modelo_arvore)
        treino_g.loc[i,alvo_mec_acao_obs] = modelo_arvore.score(X_treino_g, y_treino_g)
        teste_g.loc[i,alvo_mec_acao_obs] = modelo_arvore.score(X_teste_g, y_teste_g)
        
        modelo_arvore = DecisionTreeClassifier(max_depth=i)
        modelo_arvore.fit(X_treino_c, y_treino_c)
        modelos_c[i-profund[0]].append(modelo_arvore)
        treino_c.loc[i,alvo_mec_acao_obs] = modelo_arvore.score(X_treino_c, y_treino_c)
        teste_c.loc[i,alvo_mec_acao_obs] = modelo_arvore.score(X_teste_c, y_teste_c)
        
        modelo_arvore = DecisionTreeClassifier(max_depth=i)
        modelo_arvore.fit(X_treino_gc, y_treino_gc)
        modelos_gc[i-profund[0]].append(modelo_arvore)
        treino_gc.loc[i,alvo_mec_acao_obs] = modelo_arvore.score(X_treino_gc, y_treino_gc)
        teste_gc.loc[i,alvo_mec_acao_obs] = modelo_arvore.score(X_teste_gc, y_teste_gc)

In [186]:
from sklearn import tree
import matplotlib.pyplot as plt


def plot_tree_comp(alvo_mec_acao: str = 'acetylcholine_receptor', profundidade: int = 3, indicador: str = 'gene') -> None:
    """Gráfico de árvore dos experimentos.
      
    argumentos:
    alvo_mec_acao -- alvo do mecanismo de ação (padrão: 'adenylyl_cyclase')
    profundidade -- profundidade máxima na Árvore de Decisão (padrão: 3)
    indicador -- indicador para relação com os alvos do mecanismo de ação (padrão: 'gene')
    """
    col_idx = dados_resultados_tratados.columns.get_loc(alvo_mec_acao) - 1

    plt.style.use('seaborn-dark-palette')
    fig, ax = plt.subplots(figsize=(12,8))

    if indicador == 'gene':
        tree.plot_tree(
            modelos_g[profundidade-profund[0]][col_idx],
            ax=ax,
            fontsize=8,
            filled=True,
            rounded=True,
            feature_names=pd.DataFrame(X_treinos_g[col_idx][0]).columns,
            class_names=np.unique(y_treinos_g[col_idx]),
        )
        ax.set_title(f'Alvo do mecanismo de ação: {alvo_mec_acao} ; Acurácia no treinamento: {round(treino_g.loc[profundidade, alvo_mec_acao]*100, 2)}% ; Acurácia no teste: {round(teste_g.loc[profundidade, alvo_mec_acao]*100, 2)}%',fontdict={'fontsize':'larger','color':'black'})

    elif indicador == 'tipo celular':
        tree.plot_tree(
            modelos_c[profundidade-profund[0]][col_idx],
            ax=ax,
            fontsize=8,
            filled=True,
            rounded=True,
            feature_names=pd.DataFrame(X_treinos_c[col_idx][0]).columns,
            class_names=np.unique(y_treinos_c[col_idx]),
        )
        ax.set_title(f'Alvo do mecanismo de ação: {alvo_mec_acao} ; Acurácia no treinamento: {round(treino_c.loc[profundidade, alvo_mec_acao]*100, 2)}% ; Acurácia no teste: {round(teste_c.loc[profundidade, alvo_mec_acao]*100, 2)}%',fontdict={'fontsize':'larger','color':'black'})
        
    elif indicador == 'ambos':
        tree.plot_tree(
            modelos_gc[profundidade-profund[0]][col_idx],
            ax=ax,
            fontsize=8,
            filled=True,
            rounded=True,
            feature_names=pd.DataFrame(X_treinos_gc[col_idx][0]).columns,
            class_names=np.unique(y_treinos_gc[col_idx]),
        )
        ax.set_title(f'Alvo do mecanismo de ação: {alvo_mec_acao} ; Acurácia no treinamento: {round(treino_gc.loc[profundidade, alvo_mec_acao]*100, 2)}% ; Acurácia no teste: {round(teste_gc.loc[profundidade, alvo_mec_acao]*100, 2)}%',fontdict={'fontsize':'larger','color':'black'})
    fig.show()


interactive_plot_tree_comp = interactive(
    plot_tree_comp,
    alvo_mec_acao=alvos_mec_acao_observados,
    profundidade=list(range(profund[0],profund[1])),
    indicador=['gene','tipo celular','ambos'],
)
output = interactive_plot_tree_comp.children[-1]
interactive_plot_tree_comp

interactive(children=(Dropdown(description='alvo_mec_acao', index=3, options=('5-alpha_reductase', '11-beta-hs…

In [185]:
import numpy as np


def plot_df(alvo_m_a: str = 'acetylcholine_receptor', tratamento: str = 'com_droga') -> None:
    """DataFrame interativo de mecanismos de ação relevantes sobre um alvo e tratamento determinados.
      
    argumentos:
    alvo_m_a -- alvo do mecanismo de ação (padrão: 'adenylyl_cyclase')
    tratamento -- tratamento aplicado (padrão: com_droga)
    """    
    print('')
    print(f'Tabala de mecanismos de ação sobre o alvo: {alvo_m_a}; com tratamento: {tratamento}')
    print('Pode levar um instante para atualizar.')
    composto_to_drop = []
    for composto in dados['composto'].unique():
        if dados_resultados_tratados_completos[dados['composto'] == composto][alvo_m_a].unique() == 'NA':
            composto_to_drop.append(composto)

    compostos_relevantes = np.setdiff1d(dados['composto'].unique(), composto_to_drop)

    print('Tabela atualizada!')
    display(pd.crosstab(
        [
            dados_resultados_tratados_completos[alvo_m_a],
            dados[dados['tratamento'] == tratamento]['dose'],
            dados[dados['tratamento'] == tratamento]['tempo'],
        ],
        dados[dados['composto'].isin(compostos_relevantes)]['composto'],
    ))

interactive_plot_df = interactive(
    plot_df,
    alvo_m_a=dados_resultados_tratados_completos.loc[:,'5-alpha_reductase':].columns,
    tratamento=['com_droga','com_controle'],
)
output = interactive_plot_df.children[-1]
interactive_plot_df

interactive(children=(Dropdown(description='alvo_m_a', index=3, options=('5-alpha_reductase', '11-beta-hsd1', …